In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18562670
paper_name = 'andersen_gottschling_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [21]:
original_data = pd.read_excel('raw_data/genetics.108.089250-1.xlsx', sheet_name='Table S5', skiprows=3)

In [22]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 115 x 5


In [23]:
original_data.head()

,deletion,Number of Black 1/2 sectors,Total colonies screened,Number of Red 1/2 sectors,Total colonies screened.1
0,adk1,76,12052,4,17677
1,ahc2,9,21759,5,21747
2,apg17,9,6714,2,6730
3,apl6,14,28000,4,28036
4,asc1,13,21064,2,25780


In [24]:
original_data['gene'] = original_data['deletion'].astype(str)

In [25]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [26]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [27]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

              deletion  Number of Black 1/2 sectors  Total colonies screened  \
index_input                                                                    
101          wild type                           53                   139550   

             Number of Red 1/2 sectors  Total colonies screened.1      gene  \
index_input                                                                   
101                                  6                     106337  WILDTYPE   

                  orf  
index_input            
101          WILDTYPE  


In [28]:
original_data.set_index('orf', inplace=True)

In [29]:
original_data = original_data[['Number of Black 1/2 sectors','Total colonies screened']].astype(int)

In [30]:
original_data['data'] = original_data['Number of Black 1/2 sectors'] / original_data['Total colonies screened']

In [31]:
original_data = original_data.div(original_data.loc['WILDTYPE',:], axis=1)

In [32]:
original_data.drop(index='WILDTYPE', columns=['Number of Black 1/2 sectors','Total colonies screened'], inplace=True)

In [33]:
original_data = original_data.groupby(original_data.index).mean()

In [34]:
original_data.shape

(114, 1)

# Prepare the final dataset

In [36]:
data = original_data.copy()

In [37]:
dataset_ids = [16265]
datasets = datasets.reindex(index=dataset_ids)

In [38]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [39]:
data.head()

dataset_id,16265
data_type,value
orf,
YAL021C,23.337932
YAL047C,0.570659
YBL051C,1.168240
YBL058W,125.681092
YBR020W,1.215974


## Subset to the genes currently in SGD

In [40]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [41]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16265
,data_type,value
gene_id,orf,
19,YAL021C,23.337932
45,YAL047C,0.570659
139,YBL051C,1.168240
146,YBL058W,125.681092
216,YBR020W,1.215974


# Normalize

In [42]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [43]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [44]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            16265           
data_type             value     valuez
gene_id orf                           
19      YAL021C   23.337932   3.636207
45      YAL047C    0.570659   0.088912
139     YBL051C    1.168240   0.182020
146     YBL058W  125.681092  19.581959
216     YBR020W    1.215974   0.189457

# Print out

In [45]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [46]:
from IO.save_data_to_db3 import *

In [47]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18562670...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.21it/s]

Updating the data_modified_on field...
